In [1]:
import pandas as pd
import numpy as np
import re
from multiprocessing import Pool
import scattertext as st
import spacy
import pickle
%matplotlib inline

In [12]:
df_parsed = pd.read_pickle('df_parsed.pkl')
df_parsed.set_index('Date_',inplace=True)

In [5]:
top20froms = (lambda x: set(x[:20].index))(df_parsed['From'].value_counts())

In [ ]:
df_parsed['FromCat'] = df_parsed.From.apply(lambda x: x if x in top20froms else 'other')

nlp = spacy.load('en')

In [8]:
corpus = st.CorpusFromPandas(df_parsed.sample(10000), 
                              category_col='FromCat', 
                              text_col='Body',
                              nlp=nlp).build()

In [216]:
corpus = st.CorpusFromPandas(df_parsed.sample(10000), 
                              category_col='FromCat', 
                              text_col='Body',
                              nlp=nlp).build()

In [2]:
corpus = pickle.load(open('corpus.p','rb'))

In [219]:
# pickle.dump(corpus, open('corpus.p','wb'))

In [3]:
df_parsed_sample = corpus.get_df()

In [4]:
print(list(corpus.get_scaled_f_scores_vs_background().index[:10]))

['hou', 'enron', 'ect', 'forwarded', 'ena', 'ferc', 'dynegy', 'ees', 'kaminski', 'pdx']


In [5]:
scaled_f_scores_vs_background = corpus.get_scaled_f_scores_vs_background()

In [6]:
term_freq_df = corpus.get_term_freq_df()

In [7]:
for c in term_freq_df:
    f = c[:c.find(' freq')]
    term_freq_df[f+' fscore'] = corpus.get_scaled_f_scores(f)

In [8]:
columns = pd.Series([x[:x.find(' ')] for x in term_freq_df.columns]).drop_duplicates()

In [9]:
term_freq_df.columns = pd.MultiIndex.from_product([['freq','fscore'],columns])

In [11]:
term_freq_df.loc[:,'fscore'].sort_values('other',ascending=False).iloc[:200].style.background_gradient(subset=columns[1:])

In [45]:
# term_freq_df.loc[:,'fscore'].sort_values('other',ascending=False).iloc[:200].to_excel('fscores.xlsx')

# Entity Extraction

In [4]:
from polyglot.text import Text
import dask.dataframe as dd
from dask.multiprocessing import get

In [5]:
df_parsed_sample.shape

(10000, 22)

In [6]:
def extract_entity(blob):
    try:
        text = Text(blob)
        ents = text.entities
        entities = [' '.join(e) for e in ents]
        tags = [e.tag for e in ents]
        return list(zip(entities,tags))
    except:
        return []

In [7]:
def extract_entities(df):
    return df.Body.apply(extract_entity)

In [8]:
ddf = dd.from_pandas(df_parsed_sample, npartitions=6)

In [16]:
df_parsed_sample['entities'] = ddf.map_partitions(extract_entities, meta=pd.Series()).compute(get=get)

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

In [11]:
df_parsed_sample_with_entities = pd.read_pickle('df_parsed_sample_with_entities.pkl')

In [9]:
def get_topn_by_cat(df, cat, topncats=10):
    topncats = set((lambda x: x[:topncats].index)(df[cat].value_counts()))
    df[cat+'_'] = df[cat].apply(lambda x: x if x in topncats else 'other')
    entities_df = df.groupby(cat+'_').entities.sum().reset_index()

    entities_df_expanded = pd.DataFrame()
    for i, df_t in entities_df.iterrows():
        df_e = pd.DataFrame(df_t.entities, columns=['entity','tag'])
        df_e[cat+'_'] = df_t[cat+'_']
        entities_df_expanded = entities_df_expanded.append(df_e, ignore_index=True)
    def get_top100(df):
        df_t = df.pivot_table(index='entity',columns='tag',aggfunc=len).fillna(0).astype(int).reset_index()
        df_t.columns = ['entity','I-LOC','I-ORG','I-PER']
        df_t['All'] = df_t.select_dtypes(['int']).sum(1)
        df_t.sort_values('All', inplace=True, ascending=False)

        return df_t.iloc[:100]
    ents_by_cat = entities_df_expanded.groupby(cat+'_').apply(get_top100)
    return ents_by_cat.reset_index(level=[0]).reset_index(drop=True)

In [12]:
df_parsed_sample_with_entities['Year'] = df_parsed_sample_with_entities.Date_.dt.year.fillna(0).astype(int)
df_parsed_sample_with_entities['Month'] = df_parsed_sample_with_entities.Date_.dt.month.fillna(0).astype(int)
df_parsed_sample_with_entities['MonthYear'] = pd.to_datetime(df_parsed_sample_with_entities.Date_.dt.strftime('%Y-%m-01'))

In [13]:
ents_by_year = get_topn_by_cat(df_parsed_sample_with_entities, 'Year').set_index('Year_')
ents_by_month = get_topn_by_cat(df_parsed_sample_with_entities, 'Month').set_index('Month_')
ents_by_monthyear = get_topn_by_cat(df_parsed_sample_with_entities, 'MonthYear').set_index('MonthYear_')

In [14]:
ents_by_year.to_excel('ents_by_month.xlsx')
ents_by_monthyear.to_excel('ents_by_monthyear.xlsx')

# Sentiment

In [5]:
df_parsed_sample_with_entities.head()

,index,Date_,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,...,X-bcc,X-Folder,X-Origin,X-FileName,Body,Bcc,Cc,file,FromCat,entities
0,0,2001-12-28 17:04:54,<20611321.1075855235967.JavaMail.evans@thyme>,"Fri, 28 Dec 2001 09:04:54 -0800 (PST)",announcements.enron@enron.com,dl-ga-all_enron_houston_employees@enron.com,Metro Bus Passes and Woodlands Express Passes ...,1.0,text/plain; charset=us-ascii,7bit,...,,"\Judith_Townsend_Jan2002_1\Townsend, Judy\Inbox",Townsend-J,jtownse (Non-Privileged).pst,Enron employees not participating in contract ...,,,townsend-j/inbox/208.,other,"[(Enron, I-ORG), (Transportation, I-ORG), (Enr..."
1,1,2000-07-17 07:51:00,<20478085.1075854699318.JavaMail.evans@thyme>,"Mon, 17 Jul 2000 00:51:00 -0700 (PDT)",eric.bass@enron.com,robert.cass@enron.com,Re: Map and test - 2 Nat Gas Indices in TEST,1.0,text/plain; charset=us-ascii,7bit,...,,\Eric_Bass_Dec2000\Notes Folders\Sent,Bass-E,ebass.nsf,Am I going to be able to get these out on the ...,,,bass-e/sent/600.,eric.bass@enron.com,"[(Eric Robert B Cass, I-PER), (Torrey Moorer, ..."
2,2,2001-10-29 13:05:18,<1142282.1075852313874.JavaMail.evans@thyme>,"Mon, 29 Oct 2001 05:05:18 -0800 (PST)",jae.black@enron.com,/o=enron/ou=na/cn=recipients/cn=notesaddr/cn=a...,FW: options market update,1.0,text/plain; charset=us-ascii,7bit,...,,"\FSTURM (Non-Privileged)\Sturm, Fletcher J.\Inbox",STURM-F,FSTURM (Non-Privileged).pst,"-----Original Message-----\nFrom: \tChen, Hai ...",,,sturm-f/inbox/142.,other,"[(Chen, I-PER), (Hai, I-PER), (Tamara Jae, I-P..."
3,3,2000-12-01 09:51:00,<433013.1075842306013.JavaMail.evans@thyme>,"Fri, 1 Dec 2000 01:51:00 -0800 (PST)",pgraff@reliantenergy.com,debra.perlingiere@enron.com,Enfolio Master P&S Agreement,1.0,text/plain; charset=us-ascii,7bit,...,,\Debra_Perlingiere_Dec2000_June2001_1\Notes Fo...,PERLINGIERE-D,dperlin.nsf,"Debra,\n\nAs I explained to you, Reliant Servi...",,,perlingiere-d/all_documents/871.,other,"[(Debra, I-PER), (Reliant Services, I-ORG), (I..."
4,4,2001-01-09 14:25:00,<28341747.1075843762448.JavaMail.evans@thyme>,"Tue, 9 Jan 2001 06:25:00 -0800 (PST)",scott.stoness@enron.com,susan.mara@enron.com,FW: SCE news release -- Judge finds for SCE in...,1.0,text/plain; charset=us-ascii,7bit,...,,\Jeff_Dasovich_June2001\Notes Folders\Notes inbox,DASOVICH-J,jdasovic.nsf,I don't understand the import of this ruling.\...,"jeff.dasovich@enron.com, harry.kingerski@enron...","jeff.dasovich@enron.com, harry.kingerski@enron...",dasovich-j/notes_inbox/5848.,other,"[(Scott, I-PER), (Scott Stoness, I-PER), (HOU,..."


In [109]:
blob = df_parsed_sample_with_entities.Body.sample().iloc[0]

In [110]:
print(blob)

Audrey,

To finish up our tax return compliance we have to file returns for the Ohio cities.  I have gone through the information that you sent Dara Flinn for Limbach Company and found copies of the withholding reconciliations for some of the Ohio cities.  This was very helpful and greatly appreciated, but some of the withholding reconciliations don't include gross payroll for the year which is information that we have to have to file the city returns.  

I have attached a list of cities that we have received the withholding reconciliations with the gross payroll per the reconciliation and the tax withheld.  As you can see by the attached spreadsheet some of the cities don't have gross payroll listed since it was not on the withholding reconciliation Also included on the spreadsheet is a list of cities that had payroll in 1999, but we have not received a copy of the withholding reconciliation for those particular cities.  I need to confirm that they do not have any 2000 payroll.

					

In [111]:
text = Text(blob.replace('\n',' ').replace('>','').replace('  ',' '))

In [112]:
text.sentences

[Sentence("Audrey, To finish up our tax return compliance we have to file returns for the Ohio cities."),
 Sentence("I have gone through the information that you sent Dara Flinn for Limbach Company and found copies of the withholding reconciliations for some of the Ohio cities."),
 Sentence("This was very helpful and greatly appreciated, but some of the withholding reconciliations don't include gross payroll for the year which is information that we have to have to file the city returns."),
 Sentence("I have attached a list of cities that we have received the withholding reconciliations with the gross payroll per the reconciliation and the tax withheld."),
 Sentence("As you can see by the attached spreadsheet some of the cities don't have gross payroll listed since it was not on the withholding reconciliation Also included on the spreadsheet is a list of cities that had payroll in 1999, but we have not received a copy of the withholding reconciliation for those particular cities."),
 S

In [88]:
ents = text.entities

In [125]:
first_sentence = text.sentences[6]

print(first_sentence)

Melanie Gaar has contacted you about getting the payroll information for Enron Building Services Inc.


In [108]:
first_entity.positive_sentiment

0.5977011494252873

In [89]:
ents

[I-PER(['Sara']),
 I-PER(['James', 'Canney']),
 I-LOC(['London']),
 I-PER(['Carr']),
 I-PER(['Carr']),
 I-PER(['James']),
 I-PER(['Panus']),
 I-PER(['Stephanie']),
 I-PER(['Stephanie']),
 I-PER(['Carr']),
 I-ORG(['Enron', 'Capital']),
 I-ORG(['Trade', 'International']),
 I-PER(['James']),
 I-PER(['Stephanie']),
 I-PER(['James']),
 I-PER(['James']),
 I-ORG(['Carr', 'Futures']),
 I-PER(['Carr']),
 I-ORG(['Enron', 'Corp', '.', 'Guaranty']),
 I-PER(['Stephanie', 'Panus']),
 I-ORG(['Enron', 'Wholesale', 'Services']),
 I-LOC(['EB3803C', 'Houston', ',', 'Texas']),
 I-PER(['Cindy']),
 I-PER(['Glover']),
 I-PER(['Sheila']),
 I-PER(['Panus']),
 I-PER(['Stephanie']),
 I-PER(['Todd']),
 I-PER(['James', 'Canney']),
 I-PER(['Carr']),
 I-PER(['Cindy']),
 I-PER(['Glover']),
 I-PER(['Sheila']),
 I-PER(['Panus']),
 I-PER(['Stephanie']),
 I-PER(['Todd']),
 I-PER(['Horn']),
 I-PER(['Cindy']),
 I-PER(['Stephanie']),
 I-PER(['Todd']),
 I-PER(['Cindy']),
 I-PER(['Todd']),
 I-PER(['Cindy']),
 I-PER(['ECTRIC']

In [90]:
pos_sentiments = [e.positive_sentiment for e in ents]

In [91]:
pos_sentiments

[0.9955116696588869,
 0.998653500897666,
 0.9995511669658886,
 0.994165170556553,
 0.9851885098743267,
 0.9771095152603232,
 0.9703770197486535,
 0.9694793536804309,
 0.9663375224416517,
 0.9654398563734291,
 0.9587073608617595,
 0.9573608617594255,
 0.940754039497307,
 0.9331238779174147,
 0.9272890484739676,
 0.9241472172351886,
 0.9178635547576302,
 0.9111310592459605,
 0.9097845601436265,
 0.9066427289048474,
 0.9043985637342908,
 0.9012567324955116,
 0.8868940754039497,
 0.88016157989228,
 0.8792639138240574,
 0.8783662477558348,
 0.8774685816876122,
 0.874326750448833,
 0.86983842010772,
 0.8671454219030521,
 0.8581687612208259,
 0.8514362657091562,
 0.8505385996409336,
 0.8438061041292639,
 0.8429084380610413,
 0.8397666068222621,
 0.8388689407540395,
 0.8379712746858169,
 0.8348294434470377,
 0.8339317773788151,
 0.8330341113105925,
 0.829443447037702,
 0.8285457809694794,
 0.8258527827648114,
 0.823608617594255,
 0.822262118491921,
 0.8146319569120287,
 0.8078994614003591,
 0.

In [92]:
list(zip(ents,pos_sentiments))

[(I-PER(['Sara']), 0.9955116696588869),
 (I-PER(['James', 'Canney']), 0.998653500897666),
 (I-LOC(['London']), 0.9995511669658886),
 (I-PER(['Carr']), 0.994165170556553),
 (I-PER(['Carr']), 0.9851885098743267),
 (I-PER(['James']), 0.9771095152603232),
 (I-PER(['Panus']), 0.9703770197486535),
 (I-PER(['Stephanie']), 0.9694793536804309),
 (I-PER(['Stephanie']), 0.9663375224416517),
 (I-PER(['Carr']), 0.9654398563734291),
 (I-ORG(['Enron', 'Capital']), 0.9587073608617595),
 (I-ORG(['Trade', 'International']), 0.9573608617594255),
 (I-PER(['James']), 0.940754039497307),
 (I-PER(['Stephanie']), 0.9331238779174147),
 (I-PER(['James']), 0.9272890484739676),
 (I-PER(['James']), 0.9241472172351886),
 (I-ORG(['Carr', 'Futures']), 0.9178635547576302),
 (I-PER(['Carr']), 0.9111310592459605),
 (I-ORG(['Enron', 'Corp', '.', 'Guaranty']), 0.9097845601436265),
 (I-PER(['Stephanie', 'Panus']), 0.9066427289048474),
 (I-ORG(['Enron', 'Wholesale', 'Services']), 0.9043985637342908),
 (I-LOC(['EB3803C', 'Ho

In [17]:
def extract_entity(blob):
    try:
        text = Text(blob)
        ents = text.entities
        entities = [' '.join(e) for e in ents]
        tags = [e.tag for e in ents]
        pos_sentiments = [e.positive_sentiment for e in ents]
        neg_sentiments = [e.negative_sentiment for e in ents]
        return list(zip(entities,tags,pos_sentiments,neg_sentiments))
    except Exception as e:
        print(e)
        return []

In [18]:
def extract_entities(df):
    return df.Body.apply(extract_entity)

In [19]:
ddf = dd.from_pandas(df_parsed_sample, npartitions=6)

In [20]:
df_parsed_sample['entities'] = ddf.map_partitions(extract_entities, meta=pd.Series()).compute(get=get)

index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0


Detector is not able to detect the language reliably.


index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0


Detector is not able to detect the language reliably.


index 0 is out of bounds for axis 0 with size 0


...

Detector is not able to detect the language reliably.


index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0


Detector is not able to detect the language reliably.
Process ForkPoolWorker-24:
Process ForkPoolWorker-23:
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/ray/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ray/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ray/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ray/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ray/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/ray/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/ray/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/ray/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/ray/anaconda